In [1]:
%load_ext Cython

In [2]:
%%cython
import numpy as np

def minimumVerticalSeam(img, pixelsEnergy):
    rows = img.shape[0]
    columns = img.shape[1]
    
    seamEnergy = pixelsEnergy.copy()
    backtrack = np.zeros_like(seamEnergy, dtype=int) 

    for i in range(1, rows):
        for j in range(0, columns):
            if j == 0:
                idx = np.argmin(seamEnergy[i - 1, j:j + 1])
                backtrack[i, j] = idx + j
                minEnergy = seamEnergy[i - 1, idx + j]
            elif j == columns - 1:
                idx = np.argmin(seamEnergy[i - 1, j - 1 :j])
                backtrack[i, j] = idx + j - 1
                minEnergy = seamEnergy[i - 1, idx + j - 1]
            else:
                idx = np.argmin(seamEnergy[i - 1, j - 1:j + 1])
                backtrack[i, j] = idx + j - 1
                minEnergy = seamEnergy[i - 1, idx + j - 1]

            seamEnergy[i, j] += minEnergy
    
    initialHIndex = 0
    actualMin = seamEnergy[rows - 1, 0]
    for j in range(1, columns):
        if seamEnergy[rows-1, j]<actualMin:
            actualMin = seamEnergy[rows-1, j]
            initialHIndex = j
            

    return initialHIndex, backtrack

    


In [3]:
import ipywidgets as widgets
from ipywidgets import Image
from IPython.display import display

from imageio import imread, imwrite
from scipy.ndimage.filters import convolve
from ipywidgets.widgets import Button, IntSlider


def computeEnergy(img):
    
    k = img[0, 0].shape[0]
    verticalGradientFilter = np.array([
        [1.0, 2.0, 1.0],
        [0.0, 0.0, 0.0],
        [-1.0, -2.0, -1.0],
    ])
    verticalGradientFilter = np.stack([verticalGradientFilter] * k, axis=2)
    
    horizontalGradientFilter = np.array([
        [1.0, 0.0, -1.0],
        [2.0, 0.0, -2.0],
        [1.0, 0.0, -1.0],
    ])
    horizontalGradientFilter = np.stack([horizontalGradientFilter] * k, axis=2)

    img = img.astype('float32')
    convolved = np.absolute(convolve(img, verticalGradientFilter)) + np.absolute(convolve(img, horizontalGradientFilter))
    pixelsEnergy = convolved.sum(axis=2)

    return pixelsEnergy



def removeVerticalSeam(img):   
    rows = img.shape[0]
    columns = img.shape[1]
    pixelsEnergy = computeEnergy(img)
    j, backtrack = minimumVerticalSeam(img, pixelsEnergy)

    mask = np.ones((rows, columns), dtype=bool)

    for i in reversed(range(rows)):  #mette false sui pixel da rimuovere
        mask[i, j] = False
        j = backtrack[i, j]
        
 
    k = img[0, 0].shape[0]
        
    mask = np.stack([mask] * k, axis=2)

    img = img[mask].reshape((rows, columns - 1, k))

    return img




print("Original Photo:")
im = imread('in.jpg')
horizontal = widgets.IntSlider(min=im.shape[1]/4, max=im.shape[1], value=im.shape[1], description='Horizontal:')
display(horizontal)
buttonHResize = widgets.Button(description="Horizontal Resize")
display(buttonHResize)
vertical = widgets.IntSlider(min=im.shape[0]/4, max=im.shape[0], value=im.shape[0], description='Vertical:')
display(vertical)
buttonVResize = widgets.Button(description="Vertical Resize")
display(buttonVResize)
grad = computeEnergy(im)

imwrite('gradient.jpg', grad)





def computeHSeamCarving(b):
    img = imread('in.jpg')
    columns = img.shape[1]
    targetWidth = horizontal.value
    for i in reversed(range(columns - targetWidth)):
        img = removeVerticalSeam(img)
        print( str(i) + " seams to remove")
    imwrite('output.jpg', img)
    
    
buttonHResize.on_click(computeHSeamCarving)

def computeVSeamCarving(b):
    img = imread('in.jpg')
    rows = img.shape[0]
    targetHeight = vertical.value
    img = np.rot90(img, 1, (0, 1))
    
    
    for i in reversed(range(rows - targetHeight)):
        img = removeVerticalSeam(img)
        print( str(i) + " seams to remove")
    img = np.rot90(img, 3, (0, 1))
    imwrite('output.jpg', img)
    
    
buttonVResize.on_click(computeVSeamCarving)



Original Photo:


IntSlider(value=350, description='Horizontal:', max=350, min=87)

Button(description='Horizontal Resize', style=ButtonStyle())

IntSlider(value=218, description='Vertical:', max=218, min=54)

Button(description='Vertical Resize', style=ButtonStyle())

Lossy conversion from float32 to uint8. Range [0.0, 7890.0]. Convert image to uint8 prior to saving to suppress this warning.


110 seams to remove
109 seams to remove
108 seams to remove
107 seams to remove
106 seams to remove
105 seams to remove
104 seams to remove
103 seams to remove
102 seams to remove
101 seams to remove
100 seams to remove
99 seams to remove
98 seams to remove
97 seams to remove
96 seams to remove
95 seams to remove
94 seams to remove
93 seams to remove
92 seams to remove
91 seams to remove
90 seams to remove
89 seams to remove
88 seams to remove
87 seams to remove
86 seams to remove
85 seams to remove
84 seams to remove
83 seams to remove
82 seams to remove
81 seams to remove
80 seams to remove
79 seams to remove
78 seams to remove
77 seams to remove
76 seams to remove
75 seams to remove
74 seams to remove
73 seams to remove
72 seams to remove
71 seams to remove
70 seams to remove
69 seams to remove
68 seams to remove
67 seams to remove
66 seams to remove
65 seams to remove
64 seams to remove
63 seams to remove
62 seams to remove
61 seams to remove
60 seams to remove
59 seams to remove
5